# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID                 City      Lat       Lng  Max Temp  Humidity  \
0        0            adamstown -25.0660 -130.1015     21.77        76   
1        1             kargasok  59.0556   80.8572     -9.80        86   
2        2  san antonio de pale  -1.4014    5.6325     25.83        85   
3        3               muisne   0.6000  -80.0333     23.42        80   
4        4            sitangkai   4.6628  119.3928     28.02        78   

   Cloudiness  Wind Speed Country        Date  
0         100        3.80      PN  1731295747  
1         100        5.65      RU  1731295750  
2          94        4.29      GQ  1731295752  
3         100        3.61      EC  1731295753  
4         100        2.40      PH  1731295754

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

map_plot


# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0),:
]

# Drop null values
narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df


City_ID               City      Lat       Lng  Max Temp  Humidity  \
8          8           kedougou  12.5579  -12.1743     23.32        68   
127      127         joypur hat  25.0943   89.0217     25.84        60   
149      149               tura  25.5198   90.2201     26.34        58   
216      216            dhemaji  27.4833   94.5833     24.56        60   
300      300            brandon  27.9378  -82.2859     24.50        88   
345      345         titusville  28.6122  -80.8075     25.01        91   
346      346            mutuali -14.8706   37.0044     24.53        65   
356      356            yacuiba -22.0380  -63.6797     24.05        52   
443      443  ciudad cuauhtemoc  22.1833  -97.8333     25.06        88   
452      452     porto murtinho -21.6989  -57.8825     24.56        50   
465      465               save   8.0425    2.4860     23.53        87   
510      510      mermoz boabab  14.7065  -17.4758     24.88        83   
543      543            guankou  28.1500  113.6333     26.08        46   
557      557              simga  21.6333   81.7000     25.66        48   
558      558            yichang  30.7144  111.2847     23.61        39   

     Cloudiness  Wind Speed Country        Date  
8             0        0.00      SN  1731295760  
127           0        2.41      BD  1731297159  
149           0        0.79      IN  1731297195  
216           0        0.95      IN  1731297298  
300           0        2.57      US  1731297524  
345           0        3.09      US  1731297594  
346           0        1.72      MZ  1731297596  
356           0        2.72      BO  1731297612  
443           0        0.00      MX  1731297761  
452           0        1.91      BR  1731297775  
465           0        0.45      BJ  1731297797  
510           0        3.09      SN  1731297872  
543           0        0.42      CN  1731297927  
557           0        2.61      IN  1731297951  
558           0        1.10      CN  1731297953

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity Hotel Name
8             kedougou      SN  12.5579  -12.1743        68           
127         joypur hat      BD  25.0943   89.0217        60           
149               tura      IN  25.5198   90.2201        58           
216            dhemaji      IN  27.4833   94.5833        60           
300            brandon      US  27.9378  -82.2859        88           
345         titusville      US  28.6122  -80.8075        91           
346            mutuali      MZ -14.8706   37.0044        65           
356            yacuiba      BO -22.0380  -63.6797        52           
443  ciudad cuauhtemoc      MX  22.1833  -97.8333        88           
452     porto murtinho      BR -21.6989  -57.8825        50           
465               save      BJ   8.0425    2.4860        87           
510      mermoz boabab      SN  14.7065  -17.4758        83           
543            guankou      CN  28.1500  113.6333        46           
557              simga      IN  21.6333   81.7000        48           
558            yichang      CN  30.7144  111.2847        39

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
200
kedougou - nearest hotel: Auberge Thomas Sankara
200
joypur hat - nearest hotel: হোটেল প্রোম ইন্টারন্যাশনাল
200
tura - nearest hotel: Hotel Polo Orchid
200
dhemaji - nearest hotel: No hotel found
200
brandon - nearest hotel: Brandon Motor Lodge
200
titusville - nearest hotel: Palace Hotel
200
mutuali - nearest hotel: No hotel found
200
yacuiba - nearest hotel: Residencial Balcarce
200
ciudad cuauhtemoc - nearest hotel: Hotel Mandarin
200
porto murtinho - nearest hotel: Hotel Saladero Cuê
200
save - nearest hotel: Idadu Hôtel
200
mermoz boabab - nearest hotel: La maison blanche
200
guankou - nearest hotel: No hotel found
200
simga - nearest hotel: No hotel found
200
yichang - nearest hotel: Luck 7 Inn Yichang CBD


City Country      Lat       Lng  Humidity  \
8             kedougou      SN  12.5579  -12.1743        68   
127         joypur hat      BD  25.0943   89.0217        60   
149               tura      IN  25.5198   90.2201        58   
216            dhemaji      IN  27.4833   94.5833        60   
300            brandon      US  27.9378  -82.2859        88   
345         titusville      US  28.6122  -80.8075        91   
346            mutuali      MZ -14.8706   37.0044        65   
356            yacuiba      BO -22.0380  -63.6797        52   
443  ciudad cuauhtemoc      MX  22.1833  -97.8333        88   
452     porto murtinho      BR -21.6989  -57.8825        50   
465               save      BJ   8.0425    2.4860        87   
510      mermoz boabab      SN  14.7065  -17.4758        83   
543            guankou      CN  28.1500  113.6333        46   
557              simga      IN  21.6333   81.7000        48   
558            yichang      CN  30.7144  111.2847        39   

                     Hotel Name  
8        Auberge Thomas Sankara  
127  হোটেল প্রোম ইন্টারন্যাশনাল  
149           Hotel Polo Orchid  
216              No hotel found  
300         Brandon Motor Lodge  
345                Palace Hotel  
346              No hotel found  
356        Residencial Balcarce  
443              Hotel Mandarin  
452          Hotel Saladero Cuê  
465                 Idadu Hôtel  
510           La maison blanche  
543              No hotel found  
557              No hotel found  
558      Luck 7 Inn Yichang CBD

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot

map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)